# Data Cleaning Project: Energy Bills

## Mission

Ton objectif est de transformer le fichier qu'a récupéré la direction en un fichier exploitable.  

Pour les équipes de comptabilité, "exploitable" signifie qu'ils n'ont qu'à rechercher le nom d'une antenne en particulier, et ils peuvent obtenir l'historique de sa consommation d'énergie pour vérifier s'il y a des anomalies.  

Pour que cet historique soit intéressant, ils ont besoin d'avoir la consommation moyenne journalière (comme ça par exemple, ils peuvent comparer un mois de Février qui fait 28 jours avec un mois de janvier qui fait 31 jours, sans se dire que la diminution vient de la différence de jours dans un mois).

In [1]:
# Importer les bibliothèques et récupérer le fichier
import pandas as pd
import numpy as np

energy = pd.read_csv("Invoice_20201220.csv", delimiter="|")

# Afficher les premières lignes
energy.head()

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"


## Les tâches

In [2]:
# 1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. 
# On te demande de retirer tout autre caractère de la colonne "SITE_NAME".

# Extract SITE_NAME using regex pattern
energy['SITE_NAME'] = energy['SITE_NAME'].str.extract(r'(\d{4}[A-Za-z]{2})')

# Check for NaN and inf values in the dataframe and drop them
print(f"Missing values before cleaning : {energy['SITE_NAME'].isnull().sum()}")
mask = energy.isin([np.nan, np.inf, -np.inf]).any(axis=1)
energy = energy[~mask]
print(f"Missing values after cleaning : {energy['SITE_NAME'].isnull().sum()}")

energy.head()

Missing values before cleaning : 11
Missing values after cleaning : 0


,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"


In [3]:
# 2) On te demande de supprimer la colonne "INVOICE_NAME" qui n'apporte rien aux équipes.

energy.drop('INVOICE_NAME', axis=1, inplace=True)

In [4]:
# 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, 
# on les reçoit moins régulièrement (trimestre, semestre, année). 
# Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation 
# et la fin de la période de facturation.

energy['BEGIN'] = pd.to_datetime(energy['BEGIN'], format='%d/%m/%Y')
energy['END'] = pd.to_datetime(energy['END'], format='%d/%m/%Y')

energy['NB_DAYS'] = (energy['END'] - energy['BEGIN']).dt.days +1

energy.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,NB_DAYS
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",31
1,0001BB,2018-02-01,2018-02-28,2 211,"286,82",28
2,0001BB,2018-03-01,2018-03-31,2 450,"317,32",31
3,0001BB,2018-04-01,2018-04-30,2 431,"315,00",30
4,0001BB,2018-05-01,2018-05-31,2 574,"333,37",31


In [5]:
# 4) A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière 
# moyenne de la période.

energy['KWH_PERIOD'] = pd.to_numeric(energy['KWH_PERIOD'].str.replace(' ', ''))
energy['KWH_DAY'] = energy['KWH_PERIOD'] / energy['NB_DAYS']

energy.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,NB_DAYS,KWH_DAY
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.935484
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,78.964286
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.032258
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.033333
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.032258


In [6]:
# 5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on 
# dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.

energy['NB_BILLS'] = energy.groupby('SITE_NAME')['SITE_NAME'].transform('count')

energy.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,NB_DAYS,KWH_DAY,NB_BILLS
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.935484,35
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,78.964286,35
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.032258,35
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.033333,35
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.032258,35


In [7]:
# 6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes 
# les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. 
# Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie.

# Filter out rows with NaN or zero values
energy_filtered = energy[energy['KWH_DAY'].notna() & (energy['KWH_DAY'] != 0)]

# Calculate the coefficient of variation for each site
cv = energy_filtered.groupby('SITE_NAME')['KWH_DAY'].apply(lambda x: np.std(x) / np.mean(x)).reset_index()

# Rename the columns
cv.columns = ['SITE_NAME', 'CV_CONSUMPTION']

# Sort by descending coefficient of variation
cv = cv.sort_values('CV_CONSUMPTION', ascending=False)

# Print the resulting dataframe
cv.head()

,SITE_NAME,CV_CONSUMPTION
530,0195KO,2.941510
1454,0621KO,2.708906
318,0116BB,0.974172
1722,1081BB,0.904255
1419,0593BB,0.842247


In [8]:
# 7) Enfin, tu dois exporter ces deux fichiers obtenus pour pouvoir l'envoyer aux équipes de l'opérateur 
# et leur demander si c'est bien cela qu'ils attendaient de toi.

# Export the file calculating the coefficient of variation
cv.to_csv('cv_consumption.csv', index=False)

# Export the energy dataframe with modifications
energy.to_csv('energy_cleaned.csv', index=False)
